In [11]:
import pandas as pd
from IPython.display import display 

data = pd.read_csv('../data/vehicle_price_prediction.csv',sep=',')
display(data.head(50))





,make,model,year,mileage,engine_hp,transmission,fuel_type,drivetrain,body_type,exterior_color,interior_color,owner_count,accident_history,seller_type,condition,trim,vehicle_age,mileage_per_year,brand_popularity,price
0,Volkswagen,Jetta,2016,183903,173,Manual,Electric,RWD,Sedan,Blue,Brown,5,NaN,Dealer,Excellent,EX,9,20433.666667,0.040054,7208.52
1,Lexus,RX,2010,236643,352,Manual,Gasoline,FWD,Sedan,Silver,Beige,5,Minor,Dealer,Good,LX,15,15776.200000,0.039921,6911.81
2,Subaru,Crosstrek,2016,103199,188,Automatic,Diesel,AWD,Sedan,Silver,Beige,5,NaN,Dealer,Excellent,Touring,9,11466.555556,0.040230,11915.63
3,Cadillac,Lyriq,2016,118889,338,Manual,Gasoline,AWD,SUV,Black,Gray,3,NaN,Private,Good,Base,9,13209.888889,0.039847,25984.79
4,Toyota,Highlander,2018,204170,196,Manual,Diesel,FWD,Sedan,Red,Brown,5,Minor,Dealer,Excellent,Sport,7,29167.142857,0.039627,8151.30
5,Land Rover,Defender,2021,44907,479,Manual,Diesel,RWD,SUV,Silver,Black,5,NaN,Dealer,Excellent,Touring,4,11226.750000,0.040133,59490.99
6,Mazda,Mazda3,2018,129770,160,Automatic,Diesel,FWD,Hatchback,White,Beige,1,NaN,Dealer,Good,EX,7,18538.571429,0.040247,10822.61
7,Volkswagen,Atlas,2022,29146,172,Automatic,Gasoline,RWD,SUV,White,Black,5,NaN,Dealer,Excellent,Limited,3,9715.333333,0.040054,26934.78
8,Ram,2500,2015,169165,197,Automatic,Electric,RWD,Pickup Truck,Gray,Black,3,NaN,Private,Good,Sport,10,16916.500000,0.040148,11430.65
9,Chrysler,300,2025,5081,198,Automatic,Diesel,FWD,Sedan,Red,Beige,1,NaN,Private,Good,Sport,1,5081.000000,0.040059,30576.77
